In [1]:
import numpy as np
import pandas as pd

In [26]:
OUT_SIZE = 20
num_classes = 1
path = "../garbage_detection/"

In [15]:
df1 = pd.read_csv(path+'train_labels.csv')
df2 = pd.read_csv(path+'test_labels.csv')

In [16]:
df = pd.concat([df1, df2])
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,9e7858f8-6799-11e5-ba3c-40f2e96c8ad8.jpg,400,265,garbage,4.000000,112.000000,396.000000,253.000000
1,398faec8-6799-11e5-8dc4-40f2e96c8ad8.jpg,537,431,garbage,92.000000,310.000000,177.000000,389.000000
2,398faec8-6799-11e5-8dc4-40f2e96c8ad8.jpg,537,431,garbage,327.000000,398.000000,414.000000,430.000000
3,398faec8-6799-11e5-8dc4-40f2e96c8ad8.jpg,537,431,garbage,165.000000,130.000000,200.000000,165.000000
4,398faec8-6799-11e5-8dc4-40f2e96c8ad8.jpg,537,431,garbage,314.000000,271.000000,383.000000,329.000000
...,...,...,...,...,...,...,...,...
15,ba01cf5a-679e-11e5-b0d3-40f2e96c8ad8.jpg,636,546,garbage,103.000000,306.000000,612.000000,527.000000
16,bbd22d48-679e-11e5-b0d3-40f2e96c8ad8.jpg,618,430,garbage,117.000000,185.000000,534.000000,423.000000
17,be03cab8-679e-11e5-b0d3-40f2e96c8ad8.jpg,560,315,garbage,214.000000,84.000000,501.000000,227.000000
18,c3352676-679e-11e5-b0d3-40f2e96c8ad8.jpg,430,323,garbage,123.000000,134.000000,325.000000,253.000000


In [27]:
lbls = {}

In [28]:
for i in df.values:
    filename,width,height,label,xmin,ymin,xmax,ymax = i

    wid_r = OUT_SIZE/width
    hgt_r = OUT_SIZE/height

    a_xmin = round(wid_r*xmin)
    a_xmax = round(wid_r*xmax)
    a_ymin = round(hgt_r*ymin)
    a_ymax = round(hgt_r*ymax)
    
    if filename not in lbls:
        lbls[filename] = np.zeros((OUT_SIZE, OUT_SIZE, num_classes), dtype=np.uint8)
        
    lbls[filename][a_ymin:a_ymax,a_xmin:a_xmax] = 1

In [21]:
import pickle

In [32]:
with open('labels.pickle', 'wb') as f:
    pickle.dump(lbls, f)